In [24]:
import time
#starting time

start=time.time()

In [25]:
# libraries
import numpy as np 
import pandas as pd 
# import matplotlib.pyplot as plt
np.random.seed(32)


from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.manifold import TSNE

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import LSTM, Conv1D, MaxPooling1D, Dropout
from keras.utils.np_utils import to_categorical


# %matplotlib inline



In [26]:
df = pd.read_csv("/home/sonu/Downloads/GrammarandProductReviews.csv")


In [27]:
df.head()



,id,brand,categories,dateAdded,dateUpdated,ean,keys,manufacturer,manufacturerNumber,name,...,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.userCity,reviews.userProvince,reviews.username,upc
0,AV13O1A8GV-KLJ3akUyj,Universal Music,"Movies, Music & Books,Music,R&b,Movies & TV,Mo...",2017-07-25T00:52:42Z,2018-02-05T08:36:58Z,6.02537E+11,"602537205981,universalmusic/14331328,universal...",Universal Music Group / Cash Money,14331328,Pink Friday: Roman Reloaded Re-Up (w/dvd),...,NaN,0.0,5,https://redsky.target.com/groot-domain-api/v1/...,i love this album. it's very good. more to the...,Just Awesome,Los Angeles,NaN,Joshua,6.02537E+11
1,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor. This review was collected as part...,Good,NaN,NaN,Dorothy W,73416000391
2,AV14LG0R-jtxr-f38QfS,Lundberg,"Food,Packaged Foods,Snacks,Crackers,Snacks, Co...",2017-07-25T05:16:03Z,2018-02-05T11:27:45Z,73416000391,lundbergorganiccinnamontoastricecakes/b000fvzw...,Lundberg,574764,Lundberg Organic Cinnamon Toast Rice Cakes,...,100209113.0,NaN,5,https://www.walmart.com/reviews/product/29775278,Good flavor.,Good,NaN,NaN,Dorothy W,73416000391
3,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,113026909.0,NaN,1,https://www.walmart.com/reviews/product/43383370,I read through the reviews on here before look...,Disappointed,NaN,NaN,Rebecca,67981934427
4,AV16khLE-jtxr-f38VFn,K-Y,"Personal Care,Medicine Cabinet,Lubricant/Sperm...",2017-07-25T16:26:19Z,2018-02-05T11:25:51Z,67981934427,"kylovesensualitypleasuregel/b00u2whx8s,0679819...",K-Y,67981934427,K-Y Love Sensuality Pleasure Gel,...,171267657.0,NaN,1,https://www.walmart.com/reviews/product/43383370,My husband bought this gel for us. The gel cau...,Irritation,NaN,NaN,Walker557,67981934427


In [28]:
#histogram 
# plt.hist(df['reviews.rating'])

In [29]:
df['target'] = df['reviews.rating']<4




In [30]:
train_text, test_text, train_y, test_y = train_test_split(df['reviews.text'],df['target'],test_size = 0.2)

In [31]:
train_text.shape

(56835,)

In [32]:
MAX_NB_WORDS = 20000

# get the raw text data
texts_train = train_text.astype(str)
texts_test = test_text.astype(str)

# finally, vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS, char_level=False)
tokenizer.fit_on_texts(texts_train)
sequences = tokenizer.texts_to_sequences(texts_train)
sequences_test = tokenizer.texts_to_sequences(texts_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

/home/sonu/.local/lib/python3.6/site-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 26057 unique tokens.


In [33]:
sequences[0]

[589,
 37,
 1,
 128,
 80,
 125,
 5,
 690,
 29,
 61,
 3,
 244,
 130,
 341,
 17,
 21,
 1,
 887,
 8,
 1286,
 919,
 58,
 527,
 200,
 130,
 273,
 41,
 1626,
 804,
 94,
 117,
 3,
 19,
 72,
 612,
 110,
 44,
 85,
 180,
 1082,
 108,
 29,
 319,
 112,
 68,
 402,
 88,
 489,
 2,
 44,
 211,
 30,
 6,
 18,
 767]

In [34]:
type(tokenizer.word_index), len(tokenizer.word_index)

(dict, 26057)

In [35]:
index_to_word = dict((i, w) for w, i in tokenizer.word_index.items())

In [36]:
" ".join([index_to_word[i] for i in sequences[6]])

"i help in my kids classroom every time i help i always wipe the desks down with these wipes fast easy and man do they sterilize and clean the whole room smells clean after a good wiping i also do the mouse keyboards for the computers the teacher informed me that they don't have janitors and i am the only one cleaning the desks that makes these easy to use clorox wipes even more important to me"

In [37]:
seq_lens = [len(s) for s in sequences]
print("average length: %0.1f" % np.mean(seq_lens))
print("max length: %d" % max(seq_lens))



average length: 39.4
max length: 1034


In [38]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# plt.hist(seq_lens, bins=50);



In [39]:
# plt.hist([l for l in seq_lens if l < 200], bins=50);

In [40]:
MAX_SEQUENCE_LENGTH = 150

# pad sequences with 0s
x_train = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
x_test = pad_sequences(sequences_test, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

Shape of data tensor: (56835, 150)
Shape of data test tensor: (14209, 150)


In [41]:
y_train = train_y
y_test = test_y

y_train = to_categorical(np.asarray(y_train))
print('Shape of label tensor:', y_train.shape)



Shape of label tensor: (56835, 2)


In [42]:
from keras.layers import Dense, Input, Flatten
from keras.layers import GlobalAveragePooling1D, Embedding
from keras.models import Model

EMBEDDING_DIM = 50
N_CLASSES = 2

# input: a sequence of MAX_SEQUENCE_LENGTH integers
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')

embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

average = GlobalAveragePooling1D()(embedded_sequences)
predictions = Dense(N_CLASSES, activation='softmax')(average)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['acc'])

In [43]:
model.fit(x_train, y_train, validation_split=0.1,
          nb_epoch=10, batch_size=128)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  


Train on 51151 samples, validate on 5684 samples
Epoch 1/10
51151/51151 [==============================] - 6s 116us/step - loss: 0.3999 - acc: 0.8616 - val_loss: 0.3597 - val_acc: 0.8628
Epoch 2/10
51151/51151 [==============================] - 7s 128us/step - loss: 0.3226 - acc: 0.8720 - val_loss: 0.3048 - val_acc: 0.8774
Epoch 3/10
51151/51151 [==============================] - 6s 116us/step - loss: 0.2699 - acc: 0.8900 - val_loss: 0.2695 - val_acc: 0.8890
Epoch 4/10
51151/51151 [==============================] - 6s 113us/step - loss: 0.2388 - acc: 0.9025 - val_loss: 0.2549 - val_acc: 0.9015
Epoch 5/10
51151/51151 [==============================] - 6s 114us/step - loss: 0.2202 - acc: 0.9121 - val_loss: 0.2440 - val_acc: 0.9039
Epoch 6/10
51151/51151 [==============================] - 6s 114us/step - loss: 0.2074 - acc: 0.9176 - val_loss: 0.2404 - val_acc: 0.9061
Epoch 7/10
51151/51151 [==============================] - 6s 116us/step - loss: 0.1972 - acc: 0.9232 - val_loss: 0.2360 - v

In [44]:
output_test = model.predict(x_test)
print("test auc:", roc_auc_score(y_test,output_test[:,1]))



test auc: 0.9111201207455728


In [45]:
time.sleep(1)
end=time.time()
tot=end-start
print("Runtime of the program is",tot)

Runtime of the program is 67.99579858779907
